<a href="https://colab.research.google.com/github/wlgus9/DeepLearning_Study/blob/main/%EC%B6%94%EC%B2%9C_%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2장. 기본적인 추천 시스템

## 데이터 불러오기

In [ ]:
import pandas as pd

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols, encoding='latin-1')
users = users.set_index('user_id')
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [ ]:
i_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'IMDB_URL',
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\s', 'Comedy',
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep='|', names=i_cols, encoding='latin-1') 
movies = movies.set_index('movie_id')         
movies.head()

,title,release_date,video_release_date,IMDB_URL,unknown,Action,Adventure,Animation,Children\s,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


## 평균 평점을 기준으로 상위 5개의 영화

In [ ]:
# 사용자 - 영화 - 평점
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp'] # timestamp : 평가한 연도/날짜/시간을 숫자로 변환
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, encoding='latin-1') 
ratings = ratings.set_index('user_id')         
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


### Best-Seller

In [ ]:
# Best-Seller 추천
def recom_movie1(n_items):
    movie_sort = movie_mean.sort_values(ascending=False)[:n_items]
    recom_movies = movies.loc[movie_sort.index]
    recommendations = recom_movies['title']
    return recommendations

def recom_movie2(n_items): # recom_movie1 함수와 똑같은 기능
    return movies.loc[movie_mean.sort_values(ascending=False)[:n_items].index]['title']

movie_mean = ratings.groupby(['movie_id'])['rating'].mean()

print(recom_movie1(5))
print(recom_movie2(5))

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object
movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object


## 정확도 측정

일반적으로 RMSE 사용

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np

def RMSE(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)**2

rmse = []
for user in set(ratings.index):
    y_true = ratings.loc[user]['rating'] # 각 사용자가 평가한 모든 영화의 평점
    y_pred = movie_mean[ratings.loc[user]['movie_id']] # 각 사용자가 평가한 영화의 평균 평점
    accuracy = RMSE(y_true, y_pred) # 사용자 한 명에 대한 RMSE
    rmse.append(accuracy)

print(np.mean(rmse))    

1.3405556717741551


## 사용자 집단별 추천

In [ ]:
# 데이터 불러오기
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols, encoding='latin-1')

i_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'IMDB_URL',
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\s', 'Comedy',
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep='|', names=i_cols, encoding='latin-1') 

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, encoding='latin-1') 

In [ ]:
# timestamp 제거
ratings = ratings.drop('timestamp', axis=1)

# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# train, test set split
from sklearn.model_selection import train_test_split
x = ratings.copy() # 원본을 보존하기 위해
y = ratings['user_id'] # stratify=y : 각 user_id별로 train/test 비율 동일하게 유지
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

In [ ]:
# 정확도(RMSE) 계산
def RMSE(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)**2

# 모델별 RMSE 계산
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id']) # 사용자와 영화 짝 맞추기
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full Matrix 구하기
# Full Matrix : 사용자와 영화를 각 차원으로 하는 전체 array
# rating 데이터(사용자-영화 조합)는 전체 가능한 조합 중 일부
# 평가하지 않은 사용자-영화 조합은 NaN
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [ ]:
rating_matrix.shape
# (943, 1647) --> 사용자 : 943, 아이템 수 : 1647

(943, 1647)

In [ ]:
# 전체 평균으로 예측치를 계산하는 모델
def best_seller(user_id, movie_id):
    # train에는 없는데 test에 있는 영화가 존재할 수 있어서 try문 사용
    try:
        rating = train_mean[movie_id]
    except:
        rating = 3.0 # 3.0 : 기본값
    return rating

train_mean = x_train.groupby(['movie_id'])['rating'].mean()
score(best_seller)

1.1250019143716001

In [ ]:
# Full Matrix를 사용자 데이터와 merge
merged_ratings = pd.merge(x_train, users) # pd.merge : 동일한 이름을 가진 key가 있으면 그것을 기준으로 merge --> user_id
users = users.set_index('user_id')

# gender별 평점의 평균 계산
g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()

In [ ]:
# gender별 평균을 예측치로 돌려주는 함수
def cf_gender(user_id, movie_id):
    if movie_id in rating_matrix:
        gender = users.loc[user_id]['sex']
        if gender in g_mean[movie_id]:
            gender_rating = g_mean[movie_id][gender]
        else:
            gender_rating = 3.0
    else:
        gender_rating = 3.0
    return gender_rating

score(cf_gender)    

1.160695008185012

# 3장. 협업 필터링(CF) 추천 시스템

## 기본 CF 알고리즘

In [67]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity # 코사인 유사도 계산

matrix_dummy = rating_matrix.copy().fillna(0) # 코사인 계산 시 NaN값이 있으면 에러 발생
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy) # 모든 사용자 간의 코사인 유사도 계산
user_similarity = pd.DataFrame(user_similarity,
                               index=rating_matrix.index,
                               columns=rating_matrix.index)

In [68]:
# 주어진 영화(movie_id)의 가중평균 rating을 계산하는 함수
# 가중치 : 주어진 사용자와 다른 사용자 간의 유사도
def CF_simple(user_id, movie_id):
    if movie_id in rating_matrix:
        
        # 주어진 사용자와 다른 사용자 간의 유사도 복사
        sim_scores = user_similarity[user_id].copy()

        #주어진 영화에 대한 모든 사용자의 평점 복사
        movie_ratings = rating_matrix[movie_id].copy()

        # 주어진 영화에 대해 평가를 하지 않은 사용자의 위치 확인
        # --> 가중평균 계산에서 제외시키기 위해
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index

        # 평가하지 않은 사람(NaN) 제거
        movie_ratings = movie_ratings.dropna()

        # 주어진 영화를 평가하지 않은 사람 제거
        sim_scores = sim_scores.drop(none_rating_idx)

        # 주어진 영화를 평가한 각 사용자에 대해 평점을 유사도로 가중평균한 예측치 구하기
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

score(CF_simple) # 정확도 계산

1.096022411657435

## 이웃을 고려한 CF

이웃을 선택하는 기준  
1. KNN : 이웃의 크기를 미리 정해놓고 추천 대상 사용자와 가장 유사한 K명 선택  
2. Thresholding : 유사도의 기준을 정해놓고 이 기준을 충족시키는 사용자 선택


In [70]:
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user_id'], x_test['movie_id']) # 사용자와 영화 짝 맞추기
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

In [71]:
# Neighbor Size를 정해서 예측치 계산
def cf_knn(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_matrix:
        sim_scores = user_similarity[user_id].copy()
        movie_ratings = rating_matrix[movie_id].copy()
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.dropna()
        sim_scores = sim_scores.drop(none_rating_idx)

        # Neighbor Size가 지정되지 않은 경우
        if neighbor_size == 0:
            mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
        else: # Neighbor Size가 지정된 경우
            if len(sim_scores) > 1:
                # 평가한 사용자 수가 너무 적으면 계산에 문제가 생기기 때문에
                # 최소 2명 이상인 경우만 계산

                # 지정된 neighbor_size보다 계산 가능한 이웃의 수가 더 작은 경우도 있음
                # 그래서 둘 중 더 작은 것을 이웃의 크기로 선택
                neighbor_size = min(neighbor_size, len(sim_scores))

                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                user_idx = np.argsort(sim_scores)

                # 가장 유사도가 높은 k명의 사용자 선택
                sim_scores = sim_scores[user_idx][-neighbor_size:]

                # 선정된 k명의 사용자의 영화에 대한 평점 추출
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]

                # 추출한 평점을 유사도로 가중한 평점평균 구하기
                mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            else:
                mean_rating = 3.0
    else:
        mean_rating = 3.0
    return mean_rating

score(cf_knn, neighbor_size=30)    

1.096022411657435

## 전체 데이터로 Full Matrix, Cosine Similarity 구하기

In [72]:
rating_matirx = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')

matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,
                               index=rating_matrix.index,
                               columns=rating_matrix.index)

def recom_movie(user_id, n_items, neighbor_size=30):
    user_movie = rating_matrix.loc[user_id].copy()
    for movie in rating_matrix:
        if pd.notnull(user_movie.loc[movie]):
            # 영화 평점이 null(NaN)이 아닌 경우는 이미 평가한 경우이므로
            # 추천 리스트에서 제외하기 위해 예상 평점을 0으로
            user_movie.loc[movie] = 0
        else:
            user_movie.loc[movie] = cf_knn(user_id, movie, neighbor_size)
    movie_sort =  user_movie.sort_values(ascending=False)[:n_items]
    recom_movies = movies.loc[movie_sort.index]
    recommendations = recom_movies['title']
    return recommendations

recom_movie(user_id=2, n_items=5, neighbor_size=30)    

movie_id
1500    Prisoner of the Mountains (Kavkazsky Plennik) ...
1189                              That Old Feeling (1997)
1293                     Ayn Rand: A Sense of Life (1997)
851                              Bloody Child, The (1996)
1405                         When Night Is Falling (1995)
Name: title, dtype: object

## 최적의 이웃 크기 결정

In [73]:
rating_matirx = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')

matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity,
                               index=rating_matrix.index,
                               columns=rating_matrix.index)

for neighbor_size in [10, 20, 30, 40, 50, 60]:
    print('Neighbor Size = %d : RMSE = %.4f' % (neighbor_size, score(cf_knn, neighbor_size)))

Neighbor Size = 10 : RMSE = 1.0960
Neighbor Size = 20 : RMSE = 1.0960
Neighbor Size = 30 : RMSE = 1.0960
Neighbor Size = 40 : RMSE = 1.0960
Neighbor Size = 50 : RMSE = 1.0960
Neighbor Size = 60 : RMSE = 1.0960


## 사용자의 평가경향을 고려한 CF

In [75]:
# train 데이터의 user의 rating 평균과 영화의 평점편차 계산 
rating_mean = rating_matrix.mean(axis=1)
rating_bias = (rating_matrix.T - rating_mean).T

def CF_knn_bias(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias[movie_id].copy()
        # 현 movie에 대한 rating이 없는 사용자 삭제
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
##### (2) Neighbor size가 지정되지 않은 경우        
        if neighbor_size == 0:
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            # 편차 예측값에 현 사용자의 평균 더하기
            prediction = prediction + rating_mean[user_id]
##### (3) Neighbor size가 지정된 경우            
        else:
            # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산            
            if len(sim_scores) > 1:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

score(CF_knn_bias, 30)


0.8276704484142371

## 그 외의 CF 정확도 개선 방법

In [76]:
# train 데이터의 user의 rating 평균과 영화의 평점편차 계산 
rating_mean = rating_matrix.mean(axis=1)
rating_bias = (rating_matrix.T - rating_mean).T

##### (1)

# 사용자별 공통 평가 수 계산
rating_binary1 = np.array((rating_matrix > 0).astype(float))
rating_binary2 = rating_binary1.T
counts = np.dot(rating_binary1, rating_binary2)
counts = pd.DataFrame(counts, index=rating_matrix.index, columns=rating_matrix.index).fillna(0)

def CF_knn_bias_sig(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_bias:
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id]
        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias[movie_id]
        # 현 movie에 대한 rating이 없는 사용자 표시
        no_rating = movie_ratings.isnull()
        # 현 사용자와 다른 사용자간 공통 평가 아이템 수 가져오기 
        common_counts = counts[user_id]
        # 공통으로 평가한 영화의 수가 SIG_LEVEL보다 낮은 사용자 표시
        low_significance = common_counts < SIG_LEVEL
        # 평가를 안 하였거나, SIG_LEVEL이 기준 이하인 user 제거
        none_rating_idx = movie_ratings[no_rating | low_significance].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)
##### (2) Neighbor size가 지정되지 않은 경우        
        if neighbor_size == 0:
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            # 편차 예측값에 현 사용자의 평균 더하기
            prediction = prediction + rating_mean[user_id]
##### (3) Neighbor size가 지정된 경우            
        else:
            # 해당 영화를 평가한 사용자가 최소 MIN_RATINGS 이상인 경우에만 계산            
            if len(sim_scores) > MIN_RATINGS:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]
            else:
                prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

SIG_LEVEL = 3
MIN_RATINGS = 2
score(CF_knn_bias_sig, 30)

0.8277231412003478

## 사용자 기반 CF, 아이템 기반 CF

In [74]:
rating_matrix_t = np.transpose(rating_matrix)
matrix_dummy = rating_matrix_t.copy().fillna(0)
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=rating_matrix_t.index, columns=rating_matrix_t.index)

# 주어진 영화의 (movie_id) 가중평균 rating을 계산하는 함수, 
# 가중치는 주어진 아이템과 다른 아이템 간의 유사도(item_similarity)
def CF_IBCF(user_id, movie_id):
    if movie_id in item_similarity: # 현재 영화가 train set에 있는지 확인
        # 현재 영화와 다른 영화의 similarity 값 가져오기
        sim_scores = item_similarity[movie_id]
        # 현 사용자의 모든 rating 값 가져오기
        user_rating = rating_matrix_t[user_id]
        # 사용자가 평가하지 않은 영화 index 가져오기
        non_rating_idx = user_rating[user_rating.isnull()].index
        # 사용자가 평가하지 않은 영화 제거
        user_rating = user_rating.dropna()
        # 사용자가 평가하지 않은 영화의 similarity 값 제거
        sim_scores = sim_scores.drop(non_rating_idx)
        # 현 영화에 대한 예상 rating 계산, 가중치는 현 영화와 사용자가 평가한 영화의 유사도
        mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

# 정확도 계산
score(CF_IBCF)

1.0470443304160202